In [ ]:
#Member 1
import requests

robots_url = "https://www.amazon.com/robots.txt"
response = requests.get(robots_url)
print(response.text)

User-agent: *
Disallow: /exec/obidos/account-access-login
Disallow: /exec/obidos/change-style
Disallow: /exec/obidos/flex-sign-in
Disallow: /exec/obidos/handle-buy-box
Disallow: /exec/obidos/tg/cm/member/
Disallow: /gp/aw/help/id=sss
Disallow: /gp/cart
Disallow: /gp/flex
Disallow: /gp/product/e-mail-friend
Disallow: /gp/product/product-availability
Disallow: /gp/product/rate-this-item
Disallow: /gp/sign-in
Disallow: /gp/reader
Disallow: /gp/sitbv3/reader
Disallow: /gp/richpub/syltguides/create
Disallow: /gp/gfix
Disallow: /gp/associations/wizard.html
Disallow: /gp/dmusic/order
Disallow: /gp/legacy-handle-buy-box.html
Disallow: /gp/aws/ssop
Disallow: /gp/yourstore
Disallow: /gp/gift-central/organizer/add-wishlist
Disallow: /gp/vote
Disallow: /gp/voting/
Disallow: /gp/music/wma-pop-up
Disallow: /gp/customer-images
Disallow: /gp/richpub/listmania/createpipeline
Disallow: /gp/content-form
Disallow: /gp/pdp/invitation/invite
Disallow: /gp/customer-reviews/common/du
Disallow: /gp/customer-re

In [ ]:
#Member 1
import urllib.robotparser

# Load robots.txt from Amazon
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.amazon.com/robots.txt")
rp.read()

# Example URL tests
test_urls = [
    "https://www.amazon.com/gp/product/B09G3HRMVB",       # Product page
    "https://www.amazon.com/gp/cart",                     # Cart (blocked)
    "https://www.amazon.com/s?k=laptops",                 # Search results
    "https://www.amazon.com/gp/help/customer/contact-us", # Help page
]

user_agent = "*"

for url in test_urls:
    allowed = rp.can_fetch(user_agent, url)
    print(f"{'✅ ALLOWED' if allowed else '❌ DISALLOWED'}: {url}")


✅ ALLOWED: https://www.amazon.com/gp/product/B09G3HRMVB
❌ DISALLOWED: https://www.amazon.com/gp/cart
✅ ALLOWED: https://www.amazon.com/s?k=laptops
✅ ALLOWED: https://www.amazon.com/gp/help/customer/contact-us


In [ ]:
# Step 1: Install dependencies
!apt-get update
!apt-get install -y chromium-browser # Install chromium-browser
!pip install selenium pandas webdriver-manager # Install selenium, pandas, and webdriver-manager

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading

In [ ]:
!pip install selenium fake-useragent webdriver-manager > /dev/null
!apt-get update > /dev/null
!apt install chromium-chromedriver > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [ ]:
# ---------- IMPORT LIBRARIES ----------
import os
import time
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager
from google.colab import files

# ---------- CONFIGURATION ----------
SCRAPER_API_KEY = "0ea79f2cc5a33a5fa5c555f9cb82e331"
keyword = "laptops"
MAX_PAGES = 5
HEADLESS = True

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9"
}

# ---------- FUNCTION: Chrome Options ----------
def get_user_agent():
    try:
        return UserAgent().random
    except:
        return headers["User-Agent"]

def get_chrome_options():
    options = Options()
    if HEADLESS:
        options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={get_user_agent()}')
    return options

# ---------- FUNCTION: Scrape Product Details ----------
def scrape_amazon_product(url):
    options = get_chrome_options()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    name = soup.find(id='productTitle')
    name = name.get_text(strip=True) if name else 'N/A'

    price = soup.find('span', class_='a-offscreen')
    price = price.get_text(strip=True) if price else 'N/A'

    desc = soup.find('div', id='feature-bullets')
    if desc:
        description = ' '.join(b.get_text(strip=True) for b in desc.find_all('span') if b.get_text(strip=True))
    else:
        description = 'N/A'

    return {
        "name": name,
        "price": price,
        "description": description,
        "url": url
    }

# ---------- FUNCTION: Scrape Search Pages ----------
def get_search_result_links(keyword, max_pages=3):
    links = []
    for page in range(1, max_pages + 1):
        amazon_url = f"https://www.amazon.com/s?k={keyword}&page={page}"
        api_url = (
            f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={amazon_url}&render=true"
        )

        print(f"\n🌐 Visiting: {amazon_url}")
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"❌ Error fetching page {page}: {e}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")
        results = soup.select("div.s-result-item[data-component-type='s-search-result']")
        print(f"📦 Page {page}: {len(results)} results found")

        for product in results:
            link_elem = product.select_one("a.a-link-normal.s-no-outline")
            if link_elem:
                href = link_elem["href"]
                link = "https://www.amazon.com" + href if href.startswith("/") else href
                links.append(link)

        time.sleep(2)  # polite scraping
    return links

# ---------- MAIN ----------
def main():
    print(f"🔎 Searching for '{keyword}'...")
    product_links = get_search_result_links(keyword, MAX_PAGES)
    print(f"\n🔗 Total links found: {len(product_links)}")

    all_data = []
    for url in product_links:
        print(f"➡️ Scraping product: {url}")
        try:
            data = scrape_amazon_product(url)
            all_data.append(data)
        except Exception as e:
            print(f"❌ Error scraping {url}: {e}")

    # Save to CSV
    df = pd.DataFrame(all_data)
    filename = f"amazon_{keyword}_products_full.csv"
    df.to_csv(filename, index=False)
    print(f"\n✅ Scraping complete! Saved {len(df)} products to '{filename}'")
    #print(df.head())

    files.download(filename)

# ---------- RUN ----------
main()

🔎 Searching for 'laptops'...

🌐 Visiting: https://www.amazon.com/s?k=laptops&page=1
📦 Page 1: 16 results found

🌐 Visiting: https://www.amazon.com/s?k=laptops&page=2
📦 Page 2: 16 results found

🌐 Visiting: https://www.amazon.com/s?k=laptops&page=3
📦 Page 3: 16 results found

🌐 Visiting: https://www.amazon.com/s?k=laptops&page=4
📦 Page 4: 16 results found

🌐 Visiting: https://www.amazon.com/s?k=laptops&page=5
📦 Page 5: 16 results found

🔗 Total links found: 80
➡️ Scraping product: https://www.amazon.com/Lenovo-Computer-Dual-Core-Processor-Bluetooth/dp/B0F7HJJN7Z/ref=sr_1_1?dib=eyJ2IjoiMSJ9.5q_H7wBxJs793UANJQGtOD5pAPsviXkSccLOLQ8OC4IySXs10xmhN7cLP5JFAdx7FUovM0Jv0b6Z2_F-zf98O1K4KSwIi_EtugI-RpvGQotVfPqeBh0uR-5gJbV3Lu46lIfK6qEvzFJUiev1DUTaMyxjHlKnQxUr6G98bNK3rsv-G_oZMSFPVEvqz6KJEm70IQF9gBys3b-99frnJalb0X3jcq3hKWRHgtVH6-ezlTg.M_rdgYO5YkaUS_2rkAj3CCVgrhSsOVW4aSdAVW2SHf0&dib_tag=se&keywords=laptops&qid=1747757864&sr=8-1
➡️ Scraping product: https://www.amazon.com/A315-24P-R7VH-Display-Quad-Cor

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.9 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
import requests

url = "https://www.amazon.com"

def is_js_heavy_site(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        scripts = soup.find_all("script")
        if len(scripts) > 10 or any("application/ld+json" in str(script) for script in scripts):
            return True
        return False
    except Exception as e:
        print("Error checking site:", e)
        return None

print("JS-heavy site?" , is_js_heavy_site(url))


JS-heavy site? False


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import tempfile
import time
import requests

url = "https://www.amazon.com"

def find_rss_feeds(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    feeds = []
    for link in soup.find_all("link", type=["application/rss+xml", "application/atom+xml"]):
        href = link.get("href")
        if href:
            feeds.append(urljoin(base_url, href))
    return feeds

def find_possible_api_urls(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    api_urls = set()

    # Search for API-related href/src attributes
    for tag in soup.find_all(["a", "script", "link", "iframe", "source", "img"], href=True):
        href = tag.get("href")
        if href and ("/api/" in href.lower() or href.lower().endswith(".json")):
            api_urls.add(urljoin(base_url, href))

    for tag in soup.find_all(["a", "script", "link", "iframe", "source", "img"], src=True):
        src = tag.get("src")
        if src and ("/api/" in src.lower() or src.lower().endswith(".json")):
            api_urls.add(urljoin(base_url, src))

    # Search raw HTML for API-like patterns
    raw_api_urls = re.findall(r'https?://[^\s"\'>]+(?:/api/|\.json)[^\s"\'>]*', html, re.IGNORECASE)
    api_urls.update(raw_api_urls)

    return list(api_urls)

def run_selenium_and_extract(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Create temporary user data directory
    user_data_dir = tempfile.mkdtemp()
    options.add_argument(f"--user-data-dir={user_data_dir}")

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)  # Wait for JS to load

        html = driver.page_source
        print("✅ Page loaded successfully using Selenium")

        rss = find_rss_feeds(html, url)
        apis = find_possible_api_urls(html, url)

        print("\n📡 RSS Feeds found (if any):")
        print(rss if rss else "No obvious RSS feeds found.")

        print("\n🔗 Potential API Endpoints:")
        print(apis if apis else "No visible API endpoints found.")

        driver.quit()
    except Exception as e:
        print("❌ Selenium error:", e)

# Execute the task
run_selenium_and_extract(url)


✅ Page loaded successfully using Selenium

📡 RSS Feeds found (if any):
No obvious RSS feeds found.

🔗 Potential API Endpoints:
['https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_c_ac']


In [ ]:
from urllib.parse import urljoin

def check_robots_txt_for_feeds(url):
    try:
        robots_url = urljoin(url, "/robots.txt")
        resp = requests.get(robots_url)
        if resp.ok:
            lines = resp.text.splitlines()
            for line in lines:
                if "rss" in line.lower() or "api" in line.lower():
                    print("Mention in robots.txt:", line)
        else:
            print("robots.txt not accessible.")
    except Exception as e:
        print("Error reading robots.txt:", e)

check_robots_txt_for_feeds("https://www.amazon.com")


Mention in robots.txt: Disallow: /rss/people/*/reviews
Mention in robots.txt: Disallow: /gp/pdp/rss/*/reviews
Mention in robots.txt: Disallow: /gp/video/api


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import tempfile

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

user_data_dir = tempfile.mkdtemp()
options.add_argument(f"--user-data-dir={user_data_dir}")

try:
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    print("Page title via Selenium:", driver.title)
    driver.quit()
except Exception as e:
    print("Selenium error:", e)


Page title via Selenium: Amazon.com


In [ ]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving amazon_laptops_products_full.csv to amazon_laptops_products_full.csv


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import graphviz

# Load data from CSV
@st.cache_data
def load_data():
    return pd.read_csv("amazon_laptops_products_full.csv")

data = load_data()

# Title
st.title("🧠 Intelligent Web Crawler & Analyzer")
st.markdown("**Target Website:** Amazon (Search: Laptops)")

# Section 1 - Crawlability
st.header("🔍 Crawlability Overview")
st.metric("Crawlability Score", "70 / 100")
st.markdown("""
- ✅ Product/search pages allowed
- ❌ Cart & account pages disallowed
- ❌ Sitemap not found
- ✅ No crawl-delay detected
""")

# Section 2 - Extracted Data Table
st.header("📦 Top Extracted Product Data")
st.dataframe(data.head(10))

# Section 3 - JS/API/RSS Detection
st.header("⚙️ Technology & Access Check")
st.subheader("JavaScript-Heavy Site?")
st.success("✅ Yes – JavaScript detected via multiple <script> tags")

st.subheader("RSS Feeds?")
st.error("❌ No RSS feeds found")

st.subheader("API Endpoints?")
st.info("🔗 Few API-like links found in HTML (e.g., `/api/` or `.json`)")

# Section 4 - Crawling Recommendations
st.header("📌 Crawling Recommendations")
st.markdown("""
- Use **Selenium** or **Playwright** for dynamic pages
- No public API or RSS — use **ScraperAPI** for rendering
- Add retry delays & respect robots.txt
""")

# Section 5 - Visual Sitemap
st.header("🗺️ Visual Sitemap")
dot = graphviz.Digraph()
dot.node("Home")
dot.node("Search Results")
dot.node("Product Page")
dot.node("Cart (Blocked)")
dot.edges([("Home", "Search Results"), ("Search Results", "Product Page")])
st.graphviz_chart(dot)


Writing app.py


In [ ]:
from pyngrok import ngrok

# Set authtoken (only once per session)
ngrok.set_auth_token("2xNC5z81FSW84NRDlVNtJyChVbk_3UxQ63bTqPtZwamdYe8em")


In [ ]:
from pyngrok import ngrok

# Launch ngrok tunnel to localhost:8501
public_url = ngrok.connect("http://localhost:8501")
print("🌐 Streamlit URL:", public_url)


🌐 Streamlit URL: NgrokTunnel: "https://94ae-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import pandas as pd

st.set_page_config(page_title="Amazon Crawler Dashboard", layout="wide")

st.title("🧠 Intelligent Web Crawler & Analyzer")
st.markdown("### Website: [Amazon](https://www.amazon.com)")

# Section 1: Crawlability Summary
st.header("📋 Crawlability Summary")

crawlability = {
    "Allowed": [
        "/gp/product/B09G3HRMVB",
        "/s?k=laptops"
    ],
    "Disallowed": [
        "/gp/cart",
        "/gp/help/customer/contact-us"
    ]
}

col1, col2 = st.columns(2)
col1.metric("✅ Allowed Paths", len(crawlability["Allowed"]))
col2.metric("❌ Disallowed Paths", len(crawlability["Disallowed"]))

with st.expander("See Crawlability Rules"):
    st.json(crawlability)

# Section 2: Extracted Data
st.header("📦 Top Extracted Products")

try:
    df = pd.read_csv("amazon_laptops_products_full.csv")
    st.dataframe(df.head(10), use_container_width=True)
except:
    st.warning("Product CSV not found. Please run the scraper first.")

# Section 3: JS, RSS, and API Info
st.header("🕸️ Site Behavior Analysis")

js_heavy = True
rss_found = []
api_found = ["https://www.amazon.com/api/example.json"]

st.markdown(f"**JavaScript-heavy site:** {'✅ Yes' if js_heavy else '❌ No'}")
st.markdown("**RSS Feeds Found:** " + (', '.join(rss_found) if rss_found else "None"))
st.markdown("**API Endpoints Found:**")
st.code("\n".join(api_found) if api_found else "No public APIs detected.")

# Section 4: Tool Recommendations
st.header("🛠 Recommended Tools")

st.markdown("""
- `requests` + `BeautifulSoup` (basic HTML pages)
- `Selenium` or `Playwright` (JavaScript-rendered content)
- `ScraperAPI` or `Rotating Proxies` for large-scale scraping
- `pandas` for data storage & cleaning
""")


2025-05-20 20:11:29.432 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:11:29.434 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:11:29.562 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-20 20:11:29.564 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:11:29.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:11:29.570 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:11:29.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator()

In [ ]:
%%writefile app.py
import streamlit as st

st.title("🚀 Hello from Streamlit in Colab!")
st.write("This Streamlit app is running via ngrok.")


Overwriting app.py


In [ ]:
!pip install streamlit pyngrok


In [ ]:
from pyngrok import conf

conf.get_default().auth_token = "2xNC5z81FSW84NRDlVNtJyChVbk_3UxQ63bTqPtZwamdYe8em"


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.111.78:8501

  Stopping...


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("🌐 Public Streamlit URL:", public_url)


🌐 Public Streamlit URL: NgrokTunnel: "https://e3c5-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!ngrok config add-authtoken 2xNJZIxkc99D6Mis5rWHwGD9CPG_62Gf5EbNBfTaUiMHSiYgQ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok.kill()


In [ ]:
public_url = ngrok.connect("8501", "http")
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://af4d-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
st.set_page_config(page_title="Web Crawler & Analyzer", layout="wide")
st.title("🌐 Intelligent Web Crawler & Analyzer")
st.markdown("### Member 4: Visual & Report Designer")


2025-05-20 20:32:29.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:32:29.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:32:29.212 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:32:29.214 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:32:29.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
import streamlit as st
import pandas as pd

# 🧪 Temporary fake data to test your layout
crawled_data_df = pd.DataFrame({
    "URL": ["https://example.com", "https://openai.com"],
    "Title": ["Example Domain", "OpenAI"],
    "Keywords": [["example", "domain"], ["AI", "machine learning"]],
})

# Display with title
st.markdown("## 🔍 Analysis Results (Mock Data)")
st.dataframe(crawled_data_df)


2025-05-20 20:35:12.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:35:12.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:35:12.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:35:12.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

# Title
st.markdown("# 📊 Intelligent Web Crawler - Member 4 View")

# Tabs for layout
tab1, tab2, tab3 = st.tabs(["Overview", "Analysis Results", "Report"])

with tab1:
    st.write("Welcome! This is the frontend designed by Member 4.")
    st.write("It displays the crawled and analyzed data below.")

with tab2:
    # Mock data (replace this with real output later)
    crawled_data_df = pd.DataFrame({
        "URL": ["https://example.com", "https://openai.com"],
        "Title": ["Example Domain", "OpenAI"],
        "Keywords": [["example", "domain"], ["AI", "machine learning"]],
    })
    st.markdown("### 🔍 Analysis Results")
    st.dataframe(crawled_data_df)

with tab3:
    st.markdown("### 📥 Download Report")
    st.download_button(
        label="Download mock CSV",
        data=crawled_data_df.to_csv(index=False).encode('utf-8'),
        file_name='crawled_data.csv',
        mime='text/csv'
    )


Overwriting app.py


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.139.111.78:8502

  Stopping...


In [ ]:
from pyngrok import ngrok

# If ngrok was already running, kill the previous tunnel first
ngrok.kill()  # To avoid the "only 1 tunnel" error

# Connect to the Streamlit app
public_url = ngrok.connect(8501)
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://a608-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

st.set_page_config(page_title="IR Project", layout="wide")

st.markdown("# 🚀 Intelligent Web Crawler (Member 4 UI)")

tab1, tab2, tab3 = st.tabs(["Overview", "Analysis Results", "Report"])

with tab1:
    st.write("Frontend by Member 4. Displays and downloads results.")

with tab2:
    crawled_data_df = pd.DataFrame({
        "URL": ["https://example.com", "https://openai.com"],
        "Title": ["Example Domain", "OpenAI"],
        "Keywords": [["example", "domain"], ["AI", "machine learning"]],
    })
    st.write("### 🔍 Analysis Results")
    st.dataframe(crawled_data_df)

with tab3:
    st.write("### 📥 Export Report")
    st.download_button(
        "Download CSV",
        data=crawled_data_df.to_csv(index=False).encode("utf-8"),
        file_name="report.csv",
        mime="text/csv"
    )


Overwriting app.py


In [ ]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.111.78:8501

  Stopping...


In [ ]:
from pyngrok import ngrok

ngrok.kill()  # Important: Avoid multiple tunnel issues

public_url = ngrok.connect(8501)
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://7bc4-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import pandas as pd

st.set_page_config(page_title="IR Project - Member 4", layout="wide")

st.title("🚀 Intelligent Web Crawler (Member 4 UI)")
tab1, tab2, tab3 = st.tabs(["Overview", "Analysis Results", "Report"])

with tab1:
    st.markdown("### 📌 Project Overview")
    st.markdown("This is the Member 4 UI for displaying the crawler output.")

# Sample fake data for testing
crawled_data_df = pd.DataFrame({
    "URL": ["https://example.com", "https://openai.com"],
    "Title": ["Example Domain", "OpenAI"],
    "Keywords": [["example", "domain"], ["AI", "machine learning"]],
})

with tab2:
    st.markdown("### 🔍 Analysis Results")
    st.dataframe(crawled_data_df)

with tab3:
    st.markdown("### 📤 Download Report")
    st.download_button("Download CSV", crawled_data_df.to_csv(index=False), "results.csv", "text/csv")

    !pip install pyngrok streamlit -q

from pyngrok import ngrok
import threading

# Save the streamlit script
with open("app.py", "w") as f:
    f.write(open("/content/app.py").read())  # Replace this with actual Streamlit script or use write() directly

# Start Streamlit in a background thread
def start_streamlit():
    !streamlit run app.py

thread = threading.Thread(target=start_streamlit)
thread.start()

# Create public URL
public_url = ngrok.connect(8501)
print("🌐 Public URL:", public_url)



2025-05-20 20:41:08.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-20 20:41:08.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

🌐 Public URL: NgrokTunnel: "https://2982-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import subprocess
subprocess.Popen(["streamlit", "run", "app.py"])


<Popen: returncode: None args: ['streamlit', 'run', 'app.py']>

In [ ]:
!cat app.py


In [ ]:
!nohup streamlit run app.py &
!npx ngrok http 8501


nohup: appending output to 'nohup.out'
⠙⠹⠸⠼⠴⠦Need to install the following packages:
ngrok@5.0.0-beta.2
Ok to proceed? (y) y
ok

⠙npm error canceled
⠙npm error A complete log of this run can be found in: /root/.npm/_logs/2025-05-20T20_44_42_614Z-debug-0.log
⠙

In [ ]:
# Install ngrok only once
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin


--2025-05-20 20:45:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 35.71.179.82, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  81.0MB/s    in 0.2s    

2025-05-20 20:45:41 (81.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# Replace with your actual authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok config add-authtoken 2xNJZIxkc99D6Mis5rWHwGD9CPG_62Gf5EbNBfTaUiMHSiYgQ


NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
!nohup streamlit run app.py &
!ngrok http 8501


nohup: appending output to 'nohup.out'
Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018



In [ ]:
%%writefile crawler.py
import pandas as pd

def run_crawler():
    data = {
        "URL": [
            "https://example.com",
            "https://openai.com",
            "https://streamlit.io"
        ],
        "Title": [
            "Example Domain",
            "OpenAI – Discovering the Future of AI",
            "Streamlit — The fastest way to build data apps"
        ],
        "Status": [
            "200 OK",
            "200 OK",
            "200 OK"
        ]
    }
    return pd.DataFrame(data)


Writing crawler.py


In [ ]:
!pip install -q streamlit ngrok

# Use your ngrok authtoken if needed
# !ngrok config add-authtoken YOUR_TOKEN

# Run Streamlit with ngrok tunnel
!nohup streamlit run app.py & npx ngrok http 8501


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.8 MB/s eta 0:00:00
nohup: appending output to 'nohup.out'
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙Need to install the following packages:
ngrok@5.0.0-beta.2
Ok to proceed? (y) y
y
y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧  Stopping...
^C


In [ ]:
!yes | npx ngrok@5.0.0-beta.2 http 8501


⠙⠹⠸⠼⠴⠦⠧npm warn exec The following package was not found and will be installed: ngrok@5.0.0-beta.2
⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇^C


In [ ]:
!pip install streamlit pyngrok


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2xNJZIxkc99D6Mis5rWHwGD9CPG_62Gf5EbNBfTaUiMHSiYgQ")  # Replace with your token


In [ ]:
!nohup streamlit run app.py &


nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok

# Connect the local Streamlit server to the internet
public_url = ngrok.connect(8501)
print("🌐 Streamlit app is live at:", public_url)



🌐 Streamlit app is live at: NgrokTunnel: "https://41c4-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def is_valid_url(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def crawl_website(start_url, max_links=20):
    visited = set()
    to_visit = [start_url]
    results = []

    while to_visit and len(visited) < max_links:
        url = to_visit.pop(0)
        if url in visited:
            continue
        try:
            response = requests.get(url, timeout=5)
            visited.add(url)
            soup = BeautifulSoup(response.text, "html.parser")
            results.append({
                "url": url,
                "title": soup.title.string if soup.title else "No Title",
                "num_links": len(soup.find_all("a"))
            })
            for link_tag in soup.find_all("a", href=True):
                link = urljoin(url, link_tag['href'])
                if is_valid_url(link) and link not in visited:
                    to_visit.append(link)
        except Exception as e:
            results.append({"url": url, "error": str(e)})

    return results


In [ ]:
%%writefile crawler.py
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def is_valid_url(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def crawl_website(start_url, max_links=20):
    visited = set()
    to_visit = [start_url]
    results = []

    while to_visit and len(visited) < max_links:
        url = to_visit.pop(0)
        if url in visited:
            continue
        try:
            response = requests.get(url, timeout=5)
            visited.add(url)
            soup = BeautifulSoup(response.text, "html.parser")
            results.append({
                "url": url,
                "title": soup.title.string if soup.title else "No Title",
                "num_links": len(soup.find_all("a"))
            })
            for link_tag in soup.find_all("a", href=True):
                link = urljoin(url, link_tag['href'])
                if is_valid_url(link) and link not in visited:
                    to_visit.append(link)
        except Exception as e:
            results.append({"url": url, "error": str(e)})

    return results


Overwriting crawler.py


In [ ]:
%%writefile app.py
import streamlit as st
from crawler import crawl_website

st.set_page_config(page_title="IR Web Crawler", layout="wide")

tab1, tab2, tab3 = st.tabs(["📌 Project Overview", "🔍 Analysis Results", "📤 Download Report"])

with tab1:
    st.title("IR Project: Intelligent Web Crawler & Analyzer")
    st.write("👤 omar ehab 232125 - UI & Report Designer")
    st.markdown("### Description")
    st.write("This application allows you to run a web crawler, analyze results, and download the report.")

with tab2:
    st.subheader("Analysis Results")
    start_url = st.text_input("Enter a URL to crawl", value="https://example.com")
    max_links = st.slider("Max pages to crawl", 5, 50, 10)

    if st.button("Start Crawling"):
        with st.spinner("Crawling in progress..."):
            results = crawl_website(start_url, max_links=max_links)
        st.success(f"✅ Crawled {len(results)} pages.")
        for res in results:
            if "error" in res:
                st.error(f"{res['url']} - ❌ {res['error']}")
            else:
                st.write(f"🔗 [{res['title']}]({res['url']}) - {res['num_links']} links found")

with tab3:
    st.subheader("Download Final Report")
    st.write("📄 This section will be implemented soon...")


Overwriting app.py


In [ ]:
!nohup streamlit run app.py &


nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"🌐 App URL: {public_url}")


🌐 App URL: NgrokTunnel: "https://20a9-34-139-111-78.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
pip install streamlit beautifulsoup4 requests fpdf pandas altair


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=9ac4a56a7e5d761d21b0c3a4477fe8e2bfc7e2f287447610c91038deb8d9ff13
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
